# Adversarial vertex mixup (AVmixup)

In [1]:
import os
import sys
sys.path.append('..')
import yaml
import shutil
import argparse
import numpy as np
from tqdm import tqdm
from collections import OrderedDict

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader, random_split

from utils import *

## Parameter setting

In [2]:
gpu = '4,5,6,7'
dataset = 'cifar10'
model_type = 'wrn34-10'
checkpoint = './checkpoint/avmixup/%s/%s' % (model_type, dataset)
num_classes = 10
lr = 0.1
momentum = 0.9
weight_decay = 2e-4
batch_size = 128
total_epochs = 210
lam1 = 1
lam2 = 0.1
gamma = 2
epsilon = 8/255
alpha = 2/255
num_repeats = 10

## Inner maximization

In [3]:
def inner_max(model, xent, inputs, targets, epsilon, alpha, num_repeats):
    noise = torch.FloatTensor(inputs.shape).uniform_(-epsilon, epsilon).cuda()
    x = torch.clamp(inputs + noise, min=0, max=1)
    
    for _ in range(num_repeats):
        x.requires_grad_()
        logits = model(x)
        loss = xent(logits, targets)
        loss.backward()
        grads = x.grad.data
        x = x.detach() + alpha*torch.sign(grads).detach()
        x = torch.min(torch.max(x, inputs-epsilon), inputs+epsilon).clamp(min=0, max=1)
    return x

## Training (Outer minimization)

In [4]:
def training(epoch, model, dataloader, optimizer, num_classes, lam1, lam2,
             gamma=2, epsilon=8/255, alpha=2/255, num_repeats=10):
    model.train()
    total = 0
    total_loss = 0
    total_correct = 0
        
    xent = nn.CrossEntropyLoss()
    for idx, (inputs, targets) in enumerate(dataloader):
        inputs, targets = inputs.cuda(), targets.cuda()
        
        batch = inputs.size(0)
        x = inner_max(model, xent, inputs, targets, epsilon, alpha, num_repeats)
        pert = (x - inputs) * gamma
        x_av = torch.clamp(inputs + pert, min=0, max=1)
        onehot = torch.eye(num_classes)[targets].cuda()
        y_nat = onehot * lam1 + (onehot - 1) * ((lam1 - 1)/(num_classes - 1))
        y_adv = onehot * lam2 + (onehot - 1) * ((lam2 - 1)/(num_classes - 1))
        weight_x = torch.from_numpy(np.random.beta(1, 1, [x.size(0), 1, 1, 1])).float().cuda()
        weight_y = weight_x.view(batch, -1)
        x = weight_x * inputs + (1 - weight_x) * x_av
        y = weight_y * y_nat + (1 - weight_y) * y_adv
        
        logits = model(x)
        
        loss = torch.sum(-y * torch.log_softmax(logits, dim=1))/batch
            
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
            
        total += batch
        total_loss += loss.item()
        num_correct = logits.softmax(dim=1).argmax(dim=1).eq(targets).cpu().sum().item()
        total_correct += num_correct
        
        if idx % 100 == 0:
            print('Epoch %d [%d/%d] | loss: %.4f (avg: %.4f) | acc: %.4f (avg: %.4f) |'\
                  % (epoch, idx, len(dataloader), loss.item(), 
                     total_loss/(idx+1), 
                     num_correct/batch, total_correct/total))

In [5]:
def evaluation(epoch, model, dataloader, alpha, epsilon, num_repeats):
    model.eval()
    total_correct_nat = 0
    total_correct_adv = 0
    
    xent = nn.CrossEntropyLoss()
    with tqdm(dataloader) as pbar:
        for index, (inputs, targets) in enumerate(pbar):
            inputs, targets = inputs.cuda(), targets.cuda()
            batch = inputs.size(0)
            pbar.set_description("[eval] %d/%d" % (index+1, len(dataloader.dataset)))
            
            x = inner_max(model, xent, inputs, targets, epsilon, alpha, num_repeats)
            with torch.no_grad():
                logits_nat = model(inputs)
                logits_adv = model(x)
            total_correct_nat += torch.argmax(logits_nat.data, dim=1).eq(targets.data).cpu().sum().item()
            total_correct_adv += torch.argmax(logits_adv.data, dim=1).eq(targets.data).cpu().sum().item()
            pbar.set_postfix(OrderedDict(acc_nat=total_correct_nat / len(dataloader.dataset),
                                         acc_rob=total_correct_adv / len(dataloader.dataset)))
    
    return (total_correct_nat / len(dataloader.dataset)), (total_correct_adv / len(dataloader.dataset))

In [ ]:
os.environ['CUDA_VISIBLE_DEVICES'] = gpu
os.makedirs(checkpoint, exist_ok=True)

train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor()])
train_dataset, _ = get_dataloader(dataset, batch_size)
num_samples = len(train_dataset)
num_samples_for_train = int(num_samples * 0.98)
num_samples_for_valid = num_samples - num_samples_for_train
train_set, valid_set = random_split(train_dataset, [num_samples_for_train, num_samples_for_valid])
train_dataloader = DataLoader(train_set, batch_size=batch_size, shuffle=True, drop_last=False)
valid_dataloader = DataLoader(valid_set, batch_size=1, shuffle=True, drop_last=False)

model = nn.DataParallel(get_network(model_type, num_classes).cuda())
optimizer = optim.SGD(model.parameters(),lr=lr, momentum=momentum, weight_decay=weight_decay)

scheduler = [int(total_epochs*0.5), int(total_epochs*0.75)]
adjust_learning_rate = lr_scheduler.MultiStepLR(optimizer, scheduler, gamma=0.1)
best_acc_nat, best_acc_rob = 0, 0

for epoch in range(total_epochs):
    training(epoch, model, train_dataloader, optimizer, num_classes,
             lam1, lam2, gamma, epsilon, alpha, num_repeats)
    test_acc_nat, test_acc_rob = evaluation(epoch, model, valid_dataloader, alpha, epsilon, num_repeats)
    
    is_best = best_acc_nat < test_acc_nat and best_acc_rob < test_acc_rob
    best_acc_nat = max(best_acc_nat, test_acc_nat)
    best_acc_rob = max(best_acc_rob, test_acc_rob)
    save_checkpoint = {'state_dict': model.state_dict(),
                       'best_acc_nat': best_acc_nat,
                       'best_acc_rob': best_acc_rob,
                       'optimizer': optimizer.state_dict(),
                       'model_type': model_type,
                       'dataset': dataset}
    torch.save(save_checkpoint, os.path.join(checkpoint, 'model'))
    if is_best:
        torch.save(save_checkpoint, os.path.join(checkpoint, 'best_model'))
    adjust_learning_rate.step()


Files already downloaded and verified
Files already downloaded and verified
Epoch 0 [0/383] | loss: 2.5306 (avg: 2.5306) | acc: 0.0000 (avg: 0.0000) |
Epoch 0 [100/383] | loss: 2.2604 (avg: 2.3196) | acc: 0.1562 (avg: 0.1451) |
Epoch 0 [200/383] | loss: 2.2263 (avg: 2.2843) | acc: 0.2422 (avg: 0.1759) |
Epoch 0 [300/383] | loss: 2.2165 (avg: 2.2676) | acc: 0.2656 (avg: 0.1935) |


[eval] 1000/1000: 100%|██████████| 1000/1000 [03:22<00:00,  4.93it/s, acc_nat=0.359, acc_rob=0.253]


Epoch 1 [0/383] | loss: 2.2059 (avg: 2.2059) | acc: 0.2422 (avg: 0.2422) |
Epoch 1 [100/383] | loss: 2.2068 (avg: 2.2141) | acc: 0.2656 (avg: 0.2468) |
Epoch 1 [200/383] | loss: 2.1723 (avg: 2.2101) | acc: 0.3281 (avg: 0.2512) |
Epoch 1 [300/383] | loss: 2.1487 (avg: 2.2051) | acc: 0.3359 (avg: 0.2565) |


[eval] 1000/1000: 100%|██████████| 1000/1000 [03:19<00:00,  5.00it/s, acc_nat=0.421, acc_rob=0.283]


Epoch 2 [0/383] | loss: 2.2167 (avg: 2.2167) | acc: 0.2266 (avg: 0.2266) |
Epoch 2 [100/383] | loss: 2.2098 (avg: 2.1836) | acc: 0.2500 (avg: 0.2808) |
Epoch 2 [200/383] | loss: 2.1544 (avg: 2.1831) | acc: 0.3594 (avg: 0.2794) |
Epoch 2 [300/383] | loss: 2.2269 (avg: 2.1791) | acc: 0.2109 (avg: 0.2836) |


...

[eval] 1000/1000: 100%|██████████| 1000/1000 [03:32<00:00,  4.71it/s, acc_nat=0.416, acc_rob=0.575]


Epoch 169 [0/383] | loss: 1.5215 (avg: 1.5215) | acc: 0.9844 (avg: 0.9844) |
Epoch 169 [100/383] | loss: 1.5203 (avg: 1.5435) | acc: 0.9609 (avg: 0.9605) |
Epoch 169 [200/383] | loss: 1.5634 (avg: 1.5403) | acc: 0.9219 (avg: 0.9603) |
Epoch 169 [300/383] | loss: 1.6576 (avg: 1.5408) | acc: 0.9141 (avg: 0.9592) |


[eval] 1000/1000: 100%|██████████| 1000/1000 [03:18<00:00,  5.05it/s, acc_nat=0.481, acc_rob=0.602]


Epoch 170 [0/383] | loss: 1.5455 (avg: 1.5455) | acc: 0.9688 (avg: 0.9688) |
Epoch 170 [100/383] | loss: 1.5912 (avg: 1.5329) | acc: 0.9766 (avg: 0.9608) |
Epoch 170 [200/383] | loss: 1.6488 (avg: 1.5371) | acc: 0.9609 (avg: 0.9590) |
Epoch 170 [300/383] | loss: 1.5669 (avg: 1.5388) | acc: 0.9609 (avg: 0.9591) |


[eval] 383/1000:  38%|███▊      | 382/1000 [01:16<02:00,  5.14it/s, acc_nat=0.12, acc_rob=0.219] 